# Create Customer Order Summary
1. Join the tables Silver_Customer & Silver_Addresses & Silver_Orders
2. Retrieve the latest Address of the Customer
3. Calculate the following values
    - total_orders
    - total_items_ordered
    - total_order_amount

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW gold_customer_order_summary
AS
SELECT c.customer_id, c.customer_name, c.date_of_birth, c.telephone, c.email,
       a.address_line_1, a.city, a.state, a.postcode,
       COUNT(DISTINCT o.order_id) AS total_orders,
       SUM(CASE WHEN o.order_status IN ('Pending', 'Completed', 'Shipped') THEN o.item_quantity ELSE 0 END) AS total_items_ordered,
       SUM(CASE WHEN o.order_status IN ('Pending', 'Completed', 'Shipped') THEN o.item_quantity * o.item_price ELSE 0 END) AS total_order_amount
FROM LIVE.silver_customers AS c
JOIN LIVE.silver_addresses AS a ON c.customer_id = a.customer_id
JOIN LIVE.silver_orders AS o ON c.customer_id = o.customer_id
WHERE a.__END_AT IS NULL
GROUP BY ALL;